# Data Clean Up

In [1]:
import sqlite3
import pandas as pd
import glob
import os
import ast
from tqdm.auto import tqdm
from geopy.geocoders import Nominatim
import time
import streamlit as st

/Users/tucker/Library/Python/3.9/lib/python/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/Users/tucker/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


## Dataframes

In [2]:
folder_path = os.path.expanduser("~/DDataVizFinal/RawData")

# Function to load and label CSV in chunks
def load_and_label_chunks(file, chunksize=100_000):
    name = os.path.basename(file).lower()
    chunk_iter = pd.read_csv(file, chunksize=chunksize)
    for chunk in chunk_iter:
        if "steam" in name:
            chunk["platform"] = "steam"
        elif "ps" in name:
            chunk["platform"] = "ps"
        yield name, chunk

# Lists to collect DataFrame chunks
games_list = []
players_list = []
achievements_list = []
purchased_list = []
history_list = []

# Loop through CSV files and process in chunks
for file in tqdm(glob.glob(os.path.join(folder_path, "*.csv")), desc="Loading CSVs", mininterval=1.0):
    print(f"Loading: {file}")
    for name, chunk in load_and_label_chunks(file):
        if "games" in name and "purchased" not in name:
            games_list.append(chunk)
        elif "players" in name:
            players_list.append(chunk)
        elif "achievements" in name:
            achievements_list.append(chunk)
        elif "purchased" in name:
            purchased_list.append(chunk)
        elif "history" in name:
            history_list.append(chunk)

# Concatenate all chunks per category into full DataFrames
games_df = pd.concat(games_list, ignore_index=True)
players_df = pd.concat(players_list, ignore_index=True)
achievements_df = pd.concat(achievements_list, ignore_index=True)
purchased_df = pd.concat(purchased_list, ignore_index=True)
history_df = pd.concat(history_list, ignore_index=True)

# Optional: check shapes
# print("Games:", games_df.shape)
# print("Players:", players_df.shape)
# print("Achievements:", achievements_df.shape)
# print("Purchased:", purchased_df.shape)
# print("History:", history_df.shape)


Loading CSVs:   0%|          | 0/10 [00:00<?, ?it/s]

Loading: /Users/tucker/DDataVizFinal/RawData/achievements_ps.csv


Loading CSVs:  10%|█         | 1/10 [00:04<00:43,  4.85s/it]

Loading: /Users/tucker/DDataVizFinal/RawData/games_steam.csv
Loading: /Users/tucker/DDataVizFinal/RawData/players_ps.csv


Loading CSVs:  30%|███       | 3/10 [00:05<00:11,  1.68s/it]

...

Loading CSVs:  70%|███████   | 7/10 [00:26<00:14,  4.67s/it]

Loading: /Users/tucker/DDataVizFinal/RawData/games_ps.csv
Loading: /Users/tucker/DDataVizFinal/RawData/achievements_steam.csv


Loading CSVs:  90%|█████████ | 9/10 [00:32<00:04,  4.01s/it]

Loading: /Users/tucker/DDataVizFinal/RawData/history_ps.csv


Loading CSVs: 100%|██████████| 10/10 [01:04<00:00,  6.47s/it]


In [3]:
#Exploding the libraries
purchased_df['library']=purchased_df['library'].apply(lambda x: ast.literal_eval(x) if isinstance(x, str) else x)
purchased_df=purchased_df.explode('library').reset_index(drop=True)
purchased_df=purchased_df.rename(columns={'library': 'gameid'})
purchased_df.head()

,playerid,gameid,platform
0,76561198060698936,60,steam
1,76561198060698936,1670,steam
2,76561198060698936,3830,steam
3,76561198060698936,1600,steam
4,76561198060698936,2900,steam


In [4]:
history_df.columns = history_df.columns.str.strip()
history_df['date_acquired']=pd.to_datetime(history_df['date_acquired'])
most_recent_date=history_df['date_acquired'].max()
a_year_ago=most_recent_date-pd.Timedelta(days=365)

recent_history_df=history_df[history_df['date_acquired']>=a_year_ago]
history_df=recent_history_df.sort_values(by='date_acquired',ascending=False).reset_index(drop=True)

print(history_df.head(10))

   playerid   achievementid       date_acquired platform
0   1680436  755124_6148418 2024-12-29 04:54:45       ps
1   1291609  583652_4838391 2024-12-28 20:51:46       ps
2   1291609  583652_4838388 2024-12-28 20:51:45       ps
3   1291609  706747_5685944 2024-12-28 16:41:18       ps
4   1680436  655314_5348206 2024-12-28 08:37:23       ps
5    393284  648531_5297751 2024-12-28 05:57:44       ps
6   1152452  570737_4749466 2024-12-28 02:01:43       ps
7   1757677  137936_2181967 2024-12-28 02:01:33       ps
8   1757677  137936_2181960 2024-12-28 02:01:33       ps
9   1757677  137936_3459214 2024-12-28 01:43:34       ps


In [5]:
num_rows=history_df.shape[0]
print("Total rows:",num_rows)

Total rows: 2326079


## Cleaning NA's

### Drop Columns

In [6]:
print(list(games_df.columns))
print(list(players_df.columns))
print(list(achievements_df.columns))
print(list(purchased_df.columns))
print(list(history_df.columns))

['gameid', 'title', 'developers', 'publishers', 'genres', 'supported_languages', 'release_date', 'platform']
['playerid', 'nickname', 'country', 'platform', 'created']
['achievementid', 'gameid', 'title', 'description', 'rarity', 'platform']
['playerid', 'gameid', 'platform']
['playerid', 'achievementid', 'date_acquired', 'platform']


In [7]:
games_df=games_df.drop(columns=["developers","publishers","supported_languages"])
players_df=players_df.drop(columns=["nickname","created"])
achievements_df=achievements_df.drop(columns=["description","rarity"])

In [8]:
print(list(games_df.columns))
print(list(players_df.columns))
print(list(achievements_df.columns))
print(list(purchased_df.columns))
print(list(history_df.columns))

['gameid', 'title', 'genres', 'release_date', 'platform']
['playerid', 'country', 'platform']
['achievementid', 'gameid', 'title', 'platform']
['playerid', 'gameid', 'platform']
['playerid', 'achievementid', 'date_acquired', 'platform']


In [9]:
print(len(games_df), "games records before cleaning.")
print(len(players_df), "players records before cleaning.")
print(len(achievements_df), "achievements records before cleaning.")
print(len(purchased_df), "purchased records before cleaning.")
print(len(history_df), "history records before cleaning.")

121399 games records before cleaning.
781283 players records before cleaning.
2785590 achievements records before cleaning.
22207874 purchased records before cleaning.
2326079 history records before cleaning.


### Checking Number of NA's

In [10]:
print("Missing values before cleaning:")
print(games_df.isna().sum())

Missing values before cleaning:
gameid             0
title              3
genres          5691
release_date       0
platform           0
dtype: int64


In [11]:
print("Missing values before cleaning:")
print(players_df.isna().sum())

Missing values before cleaning:
playerid         0
country     177868
platform         0
dtype: int64


In [12]:
print("Missing values before cleaning:")
print(achievements_df.isna().sum())

Missing values before cleaning:
achievementid       0
gameid              0
title            4475
platform            0
dtype: int64


In [13]:
print("Missing values before cleaning:")
print(purchased_df.isna().sum())

Missing values before cleaning:
playerid        0
gameid      55607
platform        0
dtype: int64


In [14]:
print("Missing values before cleaning:")
print(history_df.isna().sum())

Missing values before cleaning:
playerid         0
achievementid    0
date_acquired    0
platform         0
dtype: int64


### Dropping NA's

In [15]:
games_df=games_df.dropna()
players_df=players_df.dropna()
achievements_df=achievements_df.dropna()
purchased_df=purchased_df.dropna()
history_df=history_df.dropna()
print("NAs dropped from all tables.")

NAs dropped from all tables.


In [16]:
print(len(games_df), "games records after cleaning.")
print(len(players_df), "players records after cleaning.")
print(len(achievements_df), "achievements records after cleaning.")
print(len(purchased_df), "purchased records after cleaning.")
print(len(history_df), "history records after cleaning.")

115707 games records after cleaning.
603415 players records after cleaning.
2781115 achievements records after cleaning.
22152267 purchased records after cleaning.
2326079 history records after cleaning.


## Merging Tables

In [17]:
#Merge 1
player_purchase=purchased_df.merge(
    players_df,on=["playerid","platform"],how="left")
player_purchase.head()

,playerid,gameid,platform,country
0,76561198060698936,60,steam,Russian Federation
1,76561198060698936,1670,steam,Russian Federation
2,76561198060698936,3830,steam,Russian Federation
3,76561198060698936,1600,steam,Russian Federation
4,76561198060698936,2900,steam,Russian Federation


In [18]:
#Merge 2
achievement_history=achievements_df.merge(
    history_df,on=["achievementid","platform"],how="left")
achievement_history.head()

,achievementid,gameid,title,platform,playerid,date_acquired
0,749980_6103480,749980,Platinum Cup,ps,NaN,NaT
1,749980_6103481,749980,Arcade Beginner,ps,NaN,NaT
2,749980_6103482,749980,Arcade Beginner Dirt,ps,NaN,NaT
3,749980_6103483,749980,Arcade Intermediate,ps,NaN,NaT
4,749980_6103484,749980,Arcade Advanced,ps,NaN,NaT


In [19]:
#Dropping NA's
print(len(achievement_history), "achievement records before cleaning.")
achievement_history=achievement_history.dropna()
print(len(achievement_history), "achievement records after cleaning.")

4485490 achievement records before cleaning.
2326079 achievement records after cleaning.


In [20]:
#Merge 3
game_info=achievement_history.merge(
    games_df,on=["gameid","platform"],how="left")

game_info['title']=game_info['title_y'].combine_first(game_info['title_x'])
game_info=game_info.drop(columns=['title_x', 'title_y'])

game_info.head()

,achievementid,gameid,platform,playerid,date_acquired,genres,release_date,title
0,749909_6103322,749909,ps,4482527.0,2024-11-30 07:09:59,NaN,NaN,The Music Enjoyer 1
1,749909_6103322,749909,ps,312151.0,2024-11-24 06:47:45,NaN,NaN,The Music Enjoyer 1
2,749909_6103323,749909,ps,312151.0,2024-12-13 03:39:07,NaN,NaN,The Music Enjoyer 2
3,749909_6103323,749909,ps,4482527.0,2024-11-30 07:54:12,NaN,NaN,The Music Enjoyer 2
4,749909_6103324,749909,ps,4482527.0,2024-12-01 08:10:42,NaN,NaN,The Music Enjoyer 3


In [21]:
#Dropping NA's
print(len(game_info), "game records before cleaning.")
game_info=game_info.dropna()
print(len(game_info), "game records after cleaning.")

2326079 game records before cleaning.
2090962 game records after cleaning.


In [22]:
#Adjusting to have only the dates and not the times
game_info['date_acquired']=pd.to_datetime(game_info['date_acquired'])
game_info['date_acquired']=game_info['date_acquired'].dt.strftime('%m-%d-%Y')
game_info=game_info.drop(columns=['release_date'])
game_info.rename(columns={"date_acquired": "date"},inplace=True)


In [23]:
#Final Merge
master_df = game_info.merge(
    player_purchase,
    on=["playerid", "gameid", "platform"],
    how="left"
)

print("All tables are merged into master_df")


All tables are merged into master_df


In [24]:
#Dropping NA's
print(len(master_df), "records before cleaning.")
master_df=master_df.dropna()
master_df=master_df.drop_duplicates()
print(len(master_df), "records after cleaning.")

2091053 records before cleaning.
1908295 records after cleaning.


In [25]:
master_df.head()

,achievementid,gameid,platform,playerid,date,genres,title,country
0,749388_6098572,749388,ps,2161059.0,11-27-2024,['Platformer'],Hop Up,Germany
1,749388_6098573,749388,ps,2161059.0,11-27-2024,['Platformer'],Hop Up,Germany
2,749388_6098574,749388,ps,2161059.0,11-27-2024,['Platformer'],Hop Up,Germany
3,749388_6098575,749388,ps,2161059.0,11-27-2024,['Platformer'],Hop Up,Germany
4,749388_6098576,749388,ps,2161059.0,11-27-2024,['Platformer'],Hop Up,Germany


In [26]:
geolocator=Nominatim(user_agent="geoapi_precompute")

# Get unique countries
unique_countries=master_df['country'].dropna().unique()

coords={}

print(f"Processing {len(unique_countries)} countries...\n")

for country in unique_countries:
    try:
        location=geolocator.geocode(country)
        if location:
            coords[country]={
                "latitude": location.latitude,
                "longitude": location.longitude
            }
            print(f"{country}: {location.latitude:.4f}, {location.longitude:.4f}")
        else:
            print(f"✖ No coordinates found for {country}")
        time.sleep(1)  # Stay below Nominatim rate limit
    except Exception as e:
        print(f"Error for {country}: {e}")

# Map coordinates back to dataframe
master_df["latitude"]=master_df["country"].map(lambda x: coords.get(x, {}).get("latitude"))
master_df["longitude"]=master_df["country"].map(lambda x: coords.get(x, {}).get("longitude"))

Processing 170 countries...

Germany: 51.1638, 10.4478
Canada: 61.0667, -107.9917
United States: 39.7837, -100.4459
Japan: 36.5748, 139.2394
Austria: 47.5940, 14.1246
Czechia: 49.7439, 15.3381
Poland: 52.2159, 19.1344
Hong Kong: 22.3506, 114.1849
Finland: 63.2468, 25.9209
Netherlands: 52.2435, 5.6343
France: 46.6034, 1.8883
Belgium: 50.6403, 4.6667
Italy: 42.6384, 12.6743
United Kingdom: 54.7024, -3.2766
Russian Federation: 64.6863, 97.7453
Spain: 39.3261, -4.8380
Mexico: 23.6585, -102.0077
Brazil: -10.3333, -53.2000
Portugal: 39.6622, -8.1354
Argentina: -34.9965, -64.9673
Saudi Arabia: 25.6243, 42.3528
Ireland: 52.8652, -7.9795
Switzerland: 46.7986, 8.2320
Indonesia: -2.4834, 117.8903
Australia: -24.7761, 134.7550
Greece: 38.9954, 21.9877
✖ No coordinates found for Taiwan, Province of China
New Zealand: -41.5001, 172.8344
Hungary: 47.1818, 19.5061
Sweden: 59.6750, 14.5209
Romania: 45.9852, 24.6859
Thailand: 14.8972, 100.8327
Chile: -31.7613, -71.3188
Israel: 30.8124, 34.8595
South Afr

In [27]:
master_df.head()

,achievementid,gameid,platform,playerid,date,genres,title,country,latitude,longitude
0,749388_6098572,749388,ps,2161059.0,11-27-2024,['Platformer'],Hop Up,Germany,51.163818,10.447831
1,749388_6098573,749388,ps,2161059.0,11-27-2024,['Platformer'],Hop Up,Germany,51.163818,10.447831
2,749388_6098574,749388,ps,2161059.0,11-27-2024,['Platformer'],Hop Up,Germany,51.163818,10.447831
3,749388_6098575,749388,ps,2161059.0,11-27-2024,['Platformer'],Hop Up,Germany,51.163818,10.447831
4,749388_6098576,749388,ps,2161059.0,11-27-2024,['Platformer'],Hop Up,Germany,51.163818,10.447831


In [28]:
master_df.columns=master_df.columns.str.strip()  # strip extra spaces
master_df['date']=pd.to_datetime(master_df['date'],errors='coerce')

master_df.to_csv("~/DDataVizFinal/CreatedData/test_data.csv",index=False)

## Plot Specific dataframes

### World Map

In [31]:
# ---------------------
# Get top 10 games per country (keys only)
# ---------------------
country_game_counts=master_df.groupby(['country','title']).agg(
    total_players=('playerid','nunique')
).reset_index()

top20_keys=(
    country_game_counts
    .sort_values(['country','total_players'],ascending=[True,False])
    .groupby('country')
    .head(20)[['country','title']]
)

# Merge back to get full data for top 20 games per country
world_map_data=pd.merge(master_df,top20_keys,on=['country','title'],how='inner')

print(len(master_df),"Master File Datapoints.")
print(len(world_map_data),"World Map Datapoints.")

1908295 Master File Datapoints.
231538 World Map Datapoints.


In [32]:
world_map_data['date']=pd.to_datetime(world_map_data['date'],errors='coerce')
world_map_data.to_csv("~/DDataVizFinal/CreatedData/world_map_data.csv",index=False)